## 部署准备

In [ ]:
!pip install --upgrade git+https://github.com/autodeployai/daas-client.git

In [ ]:
from daas_client import DaasClient
client = DaasClient('https://192.168.64.3:30931', 'username', 'password')

In [ ]:
project = '部署测试'
if not client.project_exists(project):
        client.create_project(project, 'deployment-test', '部署测试项目')
client.set_project(project)

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pandas as pd
from pprint import pprint
import requests

seed = 123456

iris = datasets.load_iris()
iris_target_name = 'Species'
iris_feature_names = iris.feature_names
iris_df = pd.DataFrame(iris.data, columns=iris_feature_names)
iris_df[iris_target_name] = iris.target

X, y = iris_df[iris_feature_names], iris_df[iris_target_name]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=seed)

## 部署Scikit-learn模型

In [ ]:
from sklearn.svm import SVC

model = SVC(probability=True, random_state=seed)
model.fit(X_train, y_train)

In [ ]:
publish_resp = client.publish(model,
                              name='iris',
                              mining_function='classification',
                              X_test=X_test,
                              y_test=y_test,
                              description='A SVC model')
pprint(publish_resp)

In [ ]:
test_resp = client.test(publish_resp['model_name'], model_version=publish_resp['model_version'])
pprint(test_resp)

In [ ]:
response = requests.post(test_resp['endpoint_url'],
                         headers={'Authorization': 'Bearer {token}'.format(token=test_resp['access_token'])},
                         json=test_resp['payload'],
                         verify=False)
pprint(response.json())

In [ ]:
deploy_resp = client.deploy(model_name='iris', 
                            deployment_name='iris-svc',
                            model_version=publish_resp['model_version'],
                            replicas=1)
pprint(deploy_resp)

In [ ]:
response = requests.post(deploy_resp['endpoint_url'],
                         headers={'Authorization': 'Bearer {token}'.format(token=deploy_resp['access_token'])},
                         json=deploy_resp['payload'],
                         verify=False)
pprint(response.json())

## 部署XGBoost模型

In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier(max_depth=3, objective='multi:softprob', random_state=seed)
model = model.fit(X_train, y_train)

In [ ]:
publish_resp = client.publish(model,
                              name='iris',
                              mining_function='classification',
                              X_test=X_test,
                              y_test=y_test,
                              description='A XGBClassifier model')
pprint(publish_resp)

In [ ]:
test_resp = client.test(publish_resp['model_name'], model_version=publish_resp['model_version'])
pprint(test_resp)

In [ ]:
response = requests.post(test_resp['endpoint_url'],
                         headers={'Authorization': 'Bearer {token}'.format(token=test_resp['access_token'])},
                         json=test_resp['payload'],
                         verify=False)
pprint(response.json())

## 部署LightGBM模型

In [ ]:
from lightgbm import LGBMClassifier

model = LGBMClassifier()
model = model.fit(X_train, y_train, eval_set=[(X_test, y_test)])

In [ ]:
publish_resp = client.publish(model,
                              'iris',
                              mining_function='classification',
                              X_test=X_test,
                              y_test=y_test,
                              description='A LGBMClassifier model')
pprint(publish_resp)

## 部署PySpark模型

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

spark = SparkSession.builder.getOrCreate()
df = spark.createDataFrame(iris_df)

df_train, df_test = df.randomSplit([0.7, 0.3], seed=seed)
assembler = VectorAssembler(inputCols=iris_feature_names,
                            outputCol='features')

rf = RandomForestClassifier(seed=seed).setLabelCol(iris_target_name)
pipe = Pipeline(stages=[assembler, rf])
model = pipe.fit(df_train)

In [ ]:
publish_resp = client.publish(model,
                              name='iris',
                              mining_function='classification',
                              data_test=df_test,
                              description='A RandomForestClassifier of Spark model')
pprint(publish_resp)